<a href="https://colab.research.google.com/github/codermegha/ML-project/blob/main/ml_proj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Import Libraries**

In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings("ignore")


In [12]:
application_train = pd.read_csv("/content/PS_20174392719_1491204439457_log.csv")

In [13]:
application_train.shape

(1048575, 11)

In [14]:
application_train.head(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [16]:
application_train.tail(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
1048570,95,CASH_OUT,132557.35,C1179511630,479803.00,347245.65,C435674507,484329.37,616886.72,0,0
1048571,95,PAYMENT,9917.36,C1956161225,90545.00,80627.64,M668364942,0.00,0.00,0,0
1048572,95,PAYMENT,14140.05,C2037964975,20545.00,6404.95,M1355182933,0.00,0.00,0,0
1048573,95,PAYMENT,10020.05,C1633237354,90605.00,80584.95,M1964992463,0.00,0.00,0,0
1048574,95,PAYMENT,11450.03,C1264356443,80584.95,69134.92,M677577406,0.00,0.00,0,0


# **Memory Optimization**

In [17]:
# Define your compress_memory function
def compress_memory(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

# Assuming application_train is your DataFrame
# You can replace it with your actual DataFrame name
# Replace df with application_train in the function call
application_train = compress_memory(application_train)

Memory usage of dataframe is 88.00 MB
Memory usage after optimization is: 91.80 MB
Decreased by -4.3%


In [18]:
# prompt: # Iterate over each column and print unique values

for col in application_train.columns:
    print(f"Unique values for column '{col}':")
    print(application_train[col].unique())
    print("-" * 20)

Unique values for column 'step':
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95]
--------------------
Unique values for column 'type':
['PAYMENT', 'TRANSFER', 'CASH_OUT', 'DEBIT', 'CASH_IN']
Categories (5, object): ['CASH_IN', 'CASH_OUT', 'DEBIT', 'PAYMENT', 'TRANSFER']
--------------------
Unique values for column 'amount':
[ 9839.64  1864.28   181.   ...  9917.36 14140.05 11450.03]
--------------------
Unique values for column 'nameOrig':
['C1231006815', 'C1666544295', 'C1305486145', 'C840083671', 'C2048537720', ..., 'C1179511630', 'C1956161225', 'C2037964975', 'C1633237354', 'C1264356443']
Length: 1048317
Categories (1048317, object): ['C1000001725', 'C1000004530', 'C1000008582', 'C1000009135', ...,
                               'C999989921', '

In [19]:
application_train.isnull().sum()

,0
step,0
type,0
amount,0
nameOrig,0
oldbalanceOrg,0
newbalanceOrig,0
nameDest,0
oldbalanceDest,0
newbalanceDest,0
isFraud,0


In [21]:
application_train = application_train.dropna()

In [22]:
# Identify numeric columns in the DataFrame (int, float)
# Assign application_train to null_vaules_data_processed
null_vaules_data_processed = application_train.copy()  # Create a copy to avoid modifying the original DataFrame

numeric_cols = null_vaules_data_processed.select_dtypes(include=['int64', 'float64']).columns

for column_name in numeric_cols:
    # Calculate Q1, Q3, and IQR for each numeric column
    Q1 = null_vaules_data_processed[column_name].quantile(0.25)
    Q3 = null_vaules_data_processed[column_name].quantile(0.75)
    IQR = Q3 - Q1

    # Define thresholds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Remove outliers from each numeric column
    null_vaules_data_processed = null_vaules_data_processed[
        (null_vaules_data_processed[column_name] >= lower_bound) &
        (null_vaules_data_processed[column_name] <= upper_bound)
    ]

print(application_train.shape)
print(null_vaules_data_processed.shape)

(1048575, 11)
(1048575, 11)


In [23]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

def label_encode_columns(df):
    """
    Apply label encoding to all object dtype columns in a DataFrame.

    Parameters:
    df (DataFrame): Input DataFrame.

    Returns:
    DataFrame: DataFrame with label encoding applied to object dtype columns.
    """
    label_encoded_df = df.copy()  # Create a copy to avoid modifying the original DataFrame

    for column in label_encoded_df.columns:
        if (label_encoded_df[column].dtype == 'object') or (label_encoded_df[column].dtype == 'category') :
            label_encoder = LabelEncoder()
            label_encoded_df[column] = label_encoder.fit_transform(label_encoded_df[column])
    return label_encoded_df

# Apply label encoding to all object dtype columns
data_encoded = label_encode_columns(null_vaules_data_processed)

# Check the result
data_encoded.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,3,9839.639648,125164,170136.0,160296.359375,274917,0.0,0.0,0,0
1,1,3,1864.280029,361319,21249.0,19384.720703,286716,0.0,0.0,0,0
2,1,4,181.000000,165236,181.0,0.000000,73550,0.0,0.0,1,0
3,1,1,181.000000,961662,181.0,0.000000,65464,21182.0,0.0,1,0
4,1,3,11668.139648,567915,41554.0,29885.859375,138001,0.0,0.0,0,0


**Building a base model**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load the DataFrame
# Assuming df is your DataFrame

# Data preprocessing
X = data_encoded.drop('isFraud', axis=1)
y = data_encoded['isFraud']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build Decision Tree model
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train_scaled, y_train)
dt_predictions = dt_model.predict(X_test_scaled)

print("Decision Tree Classification Report:")
print(classification_report(y_test, dt_predictions))

# Build Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(X_train_scaled, y_train)
rf_predictions = rf_model.predict(X_test_scaled)

print("\nRandom Forest Classification Report:")
print(classification_report(y_test, rf_predictions))

Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    209491
           1       0.79      0.82      0.80       224

    accuracy                           1.00    209715
   macro avg       0.90      0.91      0.90    209715
weighted avg       1.00      1.00      1.00    209715



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Function to preprocess data
def preprocess_data(df, target_column, test_size=0.2, random_state=42):
    """
    Splits the dataset into training and testing sets, and scales the features.

    Parameters:
    - df (DataFrame): The input dataset.
    - target_column (str): The name of the target column.
    - test_size (float): Proportion of test data.
    - random_state (int): Seed for reproducibility.

    Returns:
    - X_train_scaled, X_test_scaled (arrays): Scaled training and testing features.
    - y_train, y_test (arrays): Training and testing target labels.
    """
    X = df.drop(target_column, axis=1)
    y = df[target_column]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

# Function to train and evaluate a model
def train_and_evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    """
    Trains a model and evaluates it using the classification report.

    Parameters:
    - model: The machine learning model to train.
    - X_train, X_test: Training and testing features.
    - y_train, y_test: Training and testing labels.
    - model_name (str): Name of the model (for reporting).

    Returns:
    - None
    """
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(f"\n{model_name} Classification Report:")
    print(classification_report(y_test, predictions))


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Load your dataset (assuming `data_encoded` is preloaded)
# Replace `data_encoded` with your actual DataFrame
# Example: data_encoded = pd.read_csv("your_data.csv")

# Preprocess the data for Decision Tree
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(data_encoded, target_column='isFraud')

# Train and evaluate Decision Tree model
dt_model = DecisionTreeClassifier()
train_and_evaluate_model(dt_model, X_train_scaled, X_test_scaled, y_train, y_test, model_name="Decision Tree")



Decision Tree Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     29046
         1.0       0.43      0.50      0.46        18

    accuracy                           1.00     29064
   macro avg       0.71      0.75      0.73     29064
weighted avg       1.00      1.00      1.00     29064



In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Load your dataset (assuming `data_encoded` is preloaded)
# Replace `data_encoded` with your actual DataFrame
# Example: data_encoded = pd.read_csv("your_data.csv")

# Preprocess the data for Random Forest
X_train_scaled, X_test_scaled, y_train, y_test = preprocess_data(data_encoded, target_column='isFraud')

# Train and evaluate Random Forest model
rf_model = RandomForestClassifier()
train_and_evaluate_model(rf_model, X_train_scaled, X_test_scaled, y_train, y_test, model_name="Random Forest")



Random Forest Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     29046
         1.0       1.00      0.50      0.67        18

    accuracy                           1.00     29064
   macro avg       1.00      0.75      0.83     29064
weighted avg       1.00      1.00      1.00     29064



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

# Define features (X) and target variable (y)
X = data_encoded.drop(columns=['isFraud'])  # Features excluding the target variable
y = data_encoded['isFraud']  # Target variable

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Replace negative values with zero
X_train_non_negative = X_train.clip(lower=0)
X_test_non_negative = X_test.clip(lower=0)

# Feature Selection - Chi-square
chi2_selector = SelectKBest(chi2, k=4)
X_chi2_train = chi2_selector.fit_transform(X_train_non_negative, y_train)
X_chi2_test = chi2_selector.transform(X_test_non_negative)

# SMOTE Technique
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
# Decision Tree Model
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_chi2_train, y_train)
dt_pred = dt_model.predict(X_chi2_test)


In [ ]:
# Random Forest Model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_chi2_train, y_train)
rf_pred = rf_model.predict(X_chi2_test)

In [ ]:
# Bagging Model with Decision Tree as base estimator
# For older versions of scikit-learn, try using 'estimator' instead of 'base_estimator'
bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(random_state=42), random_state=42)
bagging_model.fit(X_chi2_train, y_train)
bagging_pred = bagging_model.predict(X_chi2_test)

In [ ]:
# Logistic Regression Model
log_reg_model = LogisticRegression(random_state=42)
log_reg_model.fit(X_chi2_train, y_train)
log_reg_pred = log_reg_model.predict(X_chi2_test)


In [ ]:
# Evaluate models
def evaluate_model(model_name, y_true, y_pred):
    print(f"{model_name} Classification Report:")
    print(classification_report(y_true, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    print("Accuracy Score:", accuracy_score(y_true, y_pred))

    # Evaluate Decision Tree
evaluate_model("Decision Tree", y_test, dt_pred)




Decision Tree Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     29046
         1.0       0.33      0.44      0.38        18

    accuracy                           1.00     29064
   macro avg       0.67      0.72      0.69     29064
weighted avg       1.00      1.00      1.00     29064

Confusion Matrix:
[[29030    16]
 [   10     8]]
Accuracy Score: 0.9991054225158271


In [ ]:
# Evaluate Random Forest
evaluate_model("Random Forest", y_test, rf_pred)

Random Forest Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     29046
         1.0       0.67      0.33      0.44        18

    accuracy                           1.00     29064
   macro avg       0.83      0.67      0.72     29064
weighted avg       1.00      1.00      1.00     29064

Confusion Matrix:
[[29043     3]
 [   12     6]]
Accuracy Score: 0.9994838976052849


In [ ]:
# Evaluate Bagging
evaluate_model("Bagging", y_test, bagging_pred)


Bagging Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     29046
         1.0       0.54      0.39      0.45        18

    accuracy                           1.00     29064
   macro avg       0.77      0.69      0.73     29064
weighted avg       1.00      1.00      1.00     29064

Confusion Matrix:
[[29040     6]
 [   11     7]]
Accuracy Score: 0.9994150839526562


In [ ]:
# Evaluate Logistic Regression
evaluate_model("Logistic Regression", y_test, log_reg_pred)

Logistic Regression Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     29046
         1.0       0.62      0.28      0.38        18

    accuracy                           1.00     29064
   macro avg       0.81      0.64      0.69     29064
weighted avg       1.00      1.00      1.00     29064

Confusion Matrix:
[[29043     3]
 [   13     5]]
Accuracy Score: 0.9994494907789705


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

# Define features (X) and target variable (y)
X = data_encoded.drop(columns=['isFraud'])  # Features excluding the target variable
y = data_encoded['isFraud']  # Target variable

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the models
models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'Decision Tree': DecisionTreeClassifier()
}

# Define hyperparameters for grid search
params = {
    'Logistic Regression': {'C': [0.1, 1, 10, 100]},
    'Random Forest': {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]},
    'Decision Tree': {'max_depth': [None, 10, 20], 'min_samples_split': [2, 5, 10]}
}


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

results = {}
for name, model in models.items():
    grid_search = RandomizedSearchCV(
        model,
        params[name],
        cv=3,  # Reduced folds
        scoring='accuracy',
        n_iter=50,  # Limit iterations to a subset of combinations
        n_jobs=-1  # Utilize all available cores
    )
    grid_search.fit(X_train, y_train)
    results[name] = {
        'Best Parameters': grid_search.best_params_,
        'Best Score': grid_search.best_score_
    }


In [ ]:
# Evaluate the best models on the test set
for name, model in models.items():
    model.set_params(**results[name]['Best Parameters'])
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name]['Test Accuracy'] = accuracy

In [ ]:
# Display the results
for name, result in results.items():
    print(f"Model: {name}")
    print(f"  Best Parameters: {result['Best Parameters']}")
    print(f"  Best Cross-Validation Score: {result['Best Score']:.4f}")
    print(f"  Test Accuracy: {result['Test Accuracy']:.4f}\n")

Model: Logistic Regression
  Best Parameters: {'C': 100}
  Best Cross-Validation Score: 0.9991
  Test Accuracy: 0.9995

Model: Random Forest
  Best Parameters: {'n_estimators': 50, 'max_depth': None}
  Best Cross-Validation Score: 0.9993
  Test Accuracy: 0.9997

Model: Decision Tree
  Best Parameters: {'min_samples_split': 10, 'max_depth': 10}
  Best Cross-Validation Score: 0.9991
  Test Accuracy: 0.9992



In [ ]:
import joblib
from sklearn.metrics import accuracy_score

# Assuming models are trained, evaluated, and results stored in `results`

# Save the best models (those with the highest test accuracy)
for name, result in results.items():
    # Get the correct model from the 'models' dictionary
    model = models[name]
    model.set_params(**result['Best Parameters'])  # Set best parameters
    model.fit(X_train, y_train)  # Train the model on the full training data

    # Save the model to a .pkl file
    model_filename = f"{name.replace(' ', '_')}_model.pkl"
    joblib.dump(model, model_filename)
    print(f"Model {name} saved as {model_filename}")

Model Logistic Regression saved as Logistic_Regression_model.pkl
Model Random Forest saved as Random_Forest_model.pkl
Model Decision Tree saved as Decision_Tree_model.pkl


In [ ]:
import joblib

# Assuming the model 'Decision Tree' has been trained and evaluated
model_name = 'Decision Tree'

# Retrieve the Decision Tree model from the 'models' dictionary
model = models[model_name]

# Set the best parameters for the Decision Tree model (from `results`)
model.set_params(**results[model_name]['Best Parameters'])

# Train the Decision Tree model on the full training data
model.fit(X_train, y_train)

# Save the Decision Tree model to a .pkl file
model_filename = f"{model_name.replace(' ', '_')}_model.pkl"
joblib.dump(model, model_filename)

print(f"Model {model_name} saved as {model_filename}")


Model Decision Tree saved as Decision_Tree_model.pkl


In [ ]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.7 MB/s eta 0:00:00


In [ ]:
import streamlit as st
import joblib
import numpy as np

# Load the model
model = joblib.load('Decision_Tree_model.pkl')

st.title('Decision Tree Model Deployment')

# Input fields
features = st.text_input('Enter feature values (comma-separated):')

if st.button('Predict'):
    input_features = np.array([float(x) for x in features.split(',')]).reshape(1, -1)
    prediction = model.predict(input_features)
    st.write(f'Prediction: {prediction}')


2024-12-09 07:21:42.047 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 07:21:42.369 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-12-09 07:21:42.377 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 07:21:42.390 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 07:21:42.396 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 07:21:42.398 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 07:21:42.424 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-09 07:21:42.429 Session state does not 

In [8]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00


In [9]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Sample data (replace this with real transaction data)
np.random.seed(42)
data = pd.DataFrame({
    "Transaction ID": np.arange(1, 101),
    "Customer ID": np.random.randint(1000, 2000, 100),
    "Amount": np.random.uniform(10, 500, 100),
    "Timestamp": pd.date_range(start="2024-01-01", periods=100, freq='D'),
    "Fraudulent": np.random.choice([0, 1], size=100, p=[0.9, 0.1]),
})

# Sidebar Navigation
st.sidebar.title("Credit Fraud Detection")
view_option = st.sidebar.radio("View", ["Dashboard", "Transactions", "Fraud Trends"])

# Dashboard Overview
if view_option == "Dashboard":
    st.title("Credit Fraud Detection Dashboard")

    # Summary Statistics
    total_transactions = len(data)
    total_fraud = data["Fraudulent"].sum()
    fraud_rate = (total_fraud / total_transactions) * 100

    st.subheader("Summary")
    col1, col2, col3 = st.columns(3)
    col1.metric("Total Transactions", total_transactions)
    col2.metric("Fraud Cases", total_fraud)
    col3.metric("Fraud Rate (%)", f"{fraud_rate:.2f}")

    # Transaction Amount Distribution
    st.subheader("Transaction Amount Distribution")
    fig, ax = plt.subplots()
    sns.histplot(data["Amount"], bins=20, kde=True, ax=ax)
    st.pyplot(fig)

# Transaction Table
elif view_option == "Transactions":
    st.title("Transaction Details")
    st.write("Search and filter transactions.")

    # Filter by Fraudulent Status
    filter_fraud = st.selectbox("Filter by Fraudulent Status", ["All", "Fraudulent", "Non-Fraudulent"])
    if filter_fraud == "Fraudulent":
        filtered_data = data[data["Fraudulent"] == 1]
    elif filter_fraud == "Non-Fraudulent":
        filtered_data = data[data["Fraudulent"] == 0]
    else:
        filtered_data = data

    # Display Filtered Transactions
    st.dataframe(filtered_data)

# Fraud Trends
elif view_option == "Fraud Trends":
    st.title("Fraud Trends")
    st.write("Analyze fraud patterns over time.")

    # Time-Series Plot
    fraud_trend = data.groupby(data["Timestamp"].dt.to_period("M"))["Fraudulent"].sum().reset_index()
    fraud_trend["Timestamp"] = fraud_trend["Timestamp"].astype(str)

    fig = px.line(fraud_trend, x="Timestamp", y="Fraudulent", title="Fraud Cases Over Time")
    st.plotly_chart(fig)

# Footer
st.sidebar.markdown("---")
st.sidebar.write("Credit Fraud Detection Dashboard by Streamlit")


Overwriting app.py


In [6]:
!wget -q -O - ipv4.icanhazip.com

34.46.210.225


In [7]:

! streamlit run app.py & npx localtunnel --port 8501

/bin/bash: line 1: streamlit: command not found
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋your url is: https://small-apples-push.loca.lt
^C


In [ ]:
!ngrok authtoken YOUR_AUTHTOKEN


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
